# Import

In [ ]:
import time

from sshtunnel import SSHTunnelForwarder

import pymysql

# Functions

In [2]:
def get_ec2_config():
    return {
        "host_name": "3.114.154.242",
        "ec2_port": 22,
        "ssh_username": "ec2-user",
        "ssh_pkey": "~/.ssh/vook-rails-ssh-key.pem",
        "rds_end_point": "vook-rails-db.ctutkiavfpne.ap-northeast-1.rds.amazonaws.com",
        "rds_port": 3306,
    }


def get_rds_config(port):
    return {
        "user": "root",
        "password": "",
        "port": port,
        "host": "localhost",
        "database": "vook_web_v3_production",
        "charset": "utf8mb4",
        "cursorclass": pymysql.cursors.DictCursor,
    }

In [13]:
config_ec2 = get_ec2_config()
query = """
SELECT
    a.id as knowledge_id,
    a.name as knowledge_name,
    b.name as brand_name,
    c.name as line_name
FROM
    knowledges a
LEFT JOIN
    brands b
ON
    a.brand_id = b.id
LEFT JOIN
    `lines` c
ON
    a.line_id = c.id
"""

In [14]:
# SSHトンネルの設定
with SSHTunnelForwarder(
    (config_ec2["host_name"], config_ec2["ec2_port"]),
    ssh_username=config_ec2["ssh_username"],
    ssh_pkey=config_ec2["ssh_pkey"],
    remote_bind_address=(
        config_ec2["rds_end_point"],
        config_ec2["rds_port"],
    ),
) as server:
    print(f"Local bind port: {server.local_bind_port}")
    conn = None

    try:
        conn = pymysql.connect(
            **get_rds_config(server.local_bind_port), connect_timeout=10
        )
        cursor = conn.cursor()
        # SQLクエリの実行
        cursor.execute(query)
        for row in cursor:  # column1, column2, ...は取得したいカラム名に合わせて変更してください
            print(row)

    except pymysql.MySQLError as e:
        print(f"Error connecting to MySQL: {e}")
    finally:
        if conn is not None:
            conn.close()

Local bind port: 54217
{'knowledge_id': 5, 'knowledge_name': '66前期', 'brand_name': "Levi's", 'line_name': '501'}
{'knowledge_id': 8, 'knowledge_name': '1st', 'brand_name': 'U.S.ARMY', 'line_name': 'M-65 FIELD JACKET'}
{'knowledge_id': 10, 'knowledge_name': 'BIG E', 'brand_name': "Levi's", 'line_name': '501'}
{'knowledge_id': 11, 'knowledge_name': '片面タブ', 'brand_name': "Levi's", 'line_name': '501XX'}
{'knowledge_id': 12, 'knowledge_name': '両面タブ', 'brand_name': "Levi's", 'line_name': '501XX'}
{'knowledge_id': 13, 'knowledge_name': 'ギャラ入り', 'brand_name': "Levi's", 'line_name': '501XX'}
{'knowledge_id': 15, 'knowledge_name': 'ギャラ無し', 'brand_name': "Levi's", 'line_name': '501XX'}
{'knowledge_id': 16, 'knowledge_name': 'ギャラ無し', 'brand_name': "Levi's", 'line_name': '551ZXX'}
{'knowledge_id': 17, 'knowledge_name': '501-501', 'brand_name': "Levi's", 'line_name': '501-501'}
{'knowledge_id': 19, 'knowledge_name': '66後期', 'brand_name': "Levi's", 'line_name': '501'}
{'knowledge_id': 20, 'knowledge_